In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras import Input
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LeakyReLU
from keras.layers import LSTM, SimpleRNN, GRU, Flatten, BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
PATH='/content/gdrive/My Drive/Graduation_Project/per_d200.csv'
train_data = pd.read_csv(PATH,sep=',', encoding='CP949', header=None)
#test = pd.read_csv('test.csv',sep=',', encoding='CP949')
train_data.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300
195,-1.270000e-06,0.000018,0.000035,0.000048,0.000057,0.000059,0.000055,0.000045,0.000031,0.000016,5.000000e-07,-0.000012,-0.000021,-0.000027,-2.870000e-05,-2.830000e-05,-0.000027,-2.420000e-05,-0.000022,-0.000019,-1.730000e-05,-0.000015,-0.000012,-0.000008,-0.000002,0.000006,0.000016,2.680000e-05,0.000036,0.000041,0.000041,0.000035,0.000024,0.000011,-0.000003,-0.000014,-0.000021,-0.000024,-0.000025,-0.000025,...,0.000002,7.890000e-07,-0.000002,-8.690000e-06,-0.000018,-0.000028,-0.000037,-0.000041,-0.000038,-0.000028,-0.000013,0.000003,1.710000e-05,0.000026,2.910000e-05,0.000025,0.000017,0.000007,-0.000002,-8.490000e-06,-9.900000e-06,-0.000007,3.960000e-07,0.000009,0.000017,0.000023,0.000025,0.000023,0.000017,7.560000e-06,-0.000003,-0.000012,-0.000019,-0.000023,-2.340000e-05,-0.000021,-0.000017,-0.000011,-0.000006,1
196,1.940000e-06,-0.000003,-0.000007,-0.000012,-0.000017,-0.000022,-0.000026,-0.000030,-0.000033,-0.000034,-3.360000e-05,-0.000031,-0.000028,-0.000024,-1.920000e-05,-1.380000e-05,-0.000007,2.190000e-07,0.000009,0.000018,2.690000e-05,0.000033,0.000034,0.000029,0.000021,0.000009,-0.000003,-1.080000e-05,-0.000013,-0.000007,0.000005,0.000019,0.000034,0.000044,0.000046,0.000041,0.000029,0.000013,-0.000002,-0.000015,...,-0.000012,-1.590000e-05,-0.000018,-1.900000e-05,-0.000019,-0.000020,-0.000019,-0.000018,-0.000014,-0.000009,-0.000003,0.000004,1.030000e-05,0.000014,1.600000e-05,0.000016,0.000013,0.000010,0.000007,3.640000e-06,4.820000e-08,-0.000004,-6.740000e-06,-0.000009,-0.000009,-0.000007,-0.000003,0.000002,0.000007,1.120000e-05,0.000013,0.000012,0.000009,0.000005,9.250000e-07,-0.000003,-0.000005,-0.000006,-0.000006,1
197,1.880000e-06,0.000011,0.000019,0.000024,0.000027,0.000026,0.000023,0.000018,0.000011,0.000004,-3.100000e-06,-0.000009,-0.000013,-0.000014,-1.290000e-05,-9.510000e-06,-0.000005,-2.040000e-06,-0.000002,-0.000006,-1.400000e-05,-0.000025,-0.000037,-0.000045,-0.000046,-0.000038,-0.000021,7.030000e-07,0.000023,0.000042,0.000054,0.000057,0.000054,0.000047,0.000041,0.000036,0.000034,0.000033,0.000028,0.000018,...,-0.000019,-1.320000e-05,-0.000007,2.740000e-07,0.000007,0.000014,0.000020,0.000026,0.000030,0.000033,0.000032,0.000028,2.110000e-05,0.000011,-6.960000e-07,-0.000012,-0.000022,-0.000029,-0.000031,-2.930000e-05,-2.480000e-05,-0.000019,-1.500000e-05,-0.000013,-0.000014,-0.000016,-0.000018,-0.000019,-0.000016,-8.340000e-06,0.000003,0.000015,0.000028,0.000037,4.220000e-05,0.000041,0.000035,0.000026,0.000014,1
198,2.340000e-06,-0.000032,-0.000060,-0.000079,-0.000085,-0.000080,-0.000067,-0.000048,-0.000030,-0.000016,-7.500000e-06,-0.000006,-0.000011,-0.000019,-2.780000e-05,-3.360000e-05,-0.000034,-2.850000e-05,-0.000016,0.000001,2.120000e-05,0.000040,0.000055,0.000063,0.000062,0.000054,0.000041,2.700000e-05,0.000015,0.000007,0.000005,0.000006,0.000009,0.000010,0.000008,0.000002,-0.000007,-0.000016,-0.000022,-0.000024,...,-0.000032,-1.900000e-05,-0.000006,3.710000e-06,0.000009,0.000010,0.000009,0.000006,0.000004,0.000005,0.000008,0.000012,1.510000e-05,0.000017,1.520000e-05,0.000012,0.000007,0.000004,0.000001,6.930000e-07,1.120000e-06,0.000001,3.300000e-08,-0.000003,-0.000007,-0.000011,-0.000013,-0.000011,-0.000007,-4.880000e-07,0.000006,0.000010,0.000010,0.000007,6.820000e-07,-0.000006,-0.000011,-0.000014,-0.000014,1
199,-4.320000e-07,-0.000002,-0.000005,-0.000008,-0.000013,-0.000018,-0.000023,-0.000026,-0.000027,-0.000024,-1.980000e-05,-0.000014,-0.000008,-0.000003,-5.360000e-07,-2.190000e-07,-0.000001,-3.220000e-06,-0.000004,-0.000003,-6.450000e-08,0.000005,0.000012,0.000020,0.000026,0.000031,0.000033,3.160000e-05,0.000026,0.000018,0.000007,-0.000005,-0.000015,-0.000023,-0.000028,-0.000029,-0.000027,-0.000023,-0.000019,-0.000014,...,-

In [4]:
# traindata = pd.read_csv('dataset/traindata.csv', header=None)
# testdata = pd.read_csv('dataset/testdata.csv', header=None)
# text_max_words = 144

X = train_data.iloc[:, :300]
y = train_data.iloc[:, 300]

print(X.shape)
print(y.shape)

y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.3, 
                                                    shuffle=True, 
                                                    random_state = 1001)
# 길이 맞추기
# X_train = sequence.pad_sequences(X_train, maxlen = text_max_words)
# X_test = sequence.pad_sequences(X_test, maxlen = text_max_words)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)


scaler = Normalizer().fit(X_train)
trainX = scaler.transform(X_train)

scaler = Normalizer().fit(X_test)
testT = scaler.transform(X_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))
print(X_train.shape)
print(X_test.shape)
# print(X_train)
# print(y_train)

(200, 300)
(200,)
X_train shape: (140, 300)
X_test shape: (60, 300)
y_train shape: (140, 2)
y_test shape: (60, 2)
(140, 1, 300)
(60, 1, 300)


In [5]:

batch_size = 4

# 1. define the network lstm_model1
model = Sequential()
# model.add(Embedding(32, 32))
model.add(LSTM(512, input_dim = 300, return_sequences=True))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())

model.add(LSTM(256, return_sequences=True))  
# model.add(LSTM(256, input_shape = (1, 43)))  
model.add(LeakyReLU(alpha=0.1))
# model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True))
# model.add(Activation('relu'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())

model.add(LSTM(32))
# model.add(Activation('relu'))
model.add(LeakyReLU(alpha=0.1))
# model.add(Dropout(0.1))

# fully connected layer
"""
model.add(Flatten())
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(32))
model.add(LeakyReLU(alpha=0.1))
"""

model.add(Dense(2))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 512)         1665024   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, None, 512)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, None, 512)         2048      
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         787456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, None, 256)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 256)         1024      
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 128)         1

In [6]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('/content/gdrive/My Drive/Graduation_Project/LSTM_pur_v2.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=200, callbacks=[es, mc], validation_split = 0.2)

Epoch 1/200
28/28 [==============================] - 37s 96ms/step - loss: 0.6904 - acc: 0.5265 - val_loss: 0.6931 - val_acc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.50000, saving model to /content/gdrive/My Drive/Graduation_Project/LSTM_pur_v2.h5
Epoch 2/200
28/28 [==============================] - 1s 19ms/step - loss: 0.6201 - acc: 0.7026 - val_loss: 0.6931 - val_acc: 0.5357

Epoch 00002: val_acc improved from 0.50000 to 0.53571, saving model to /content/gdrive/My Drive/Graduation_Project/LSTM_pur_v2.h5
Epoch 3/200
28/28 [==============================] - 1s 19ms/step - loss: 0.5609 - acc: 0.7395 - val_loss: 0.6929 - val_acc: 0.6071

Epoch 00003: val_acc improved from 0.53571 to 0.60714, saving model to /content/gdrive/My Drive/Graduation_Project/LSTM_pur_v2.h5
Epoch 4/200
28/28 [==============================] - 1s 18ms/step - loss: 0.5369 - acc: 0.7814 - val_loss: 0.6927 - val_acc: 0.5000

Epoch 00004: val_acc did not improve from 0.60714
Epoch 5/200
28/28 [===========

In [7]:
from keras.models import load_model
loaded_model = load_model('/content/gdrive/My Drive/Graduation_Project/LSTM_pur_v2.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

2/2 [==============================] - 2s 11ms/step - loss: 0.6663 - acc: 0.6333

 테스트 정확도: 0.6333


In [8]:
# Plot the loss and accuracy curves for training and validation 
import matplotlib.pyplot as plt
plt.plot(history.history['val_loss'], color='b', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [9]:
from keras.models import load_model
loaded_model = load_model('/content/gdrive/My Drive/Graduation_Project/LSTM_pur_v2.h5')

In [10]:
from sklearn.metrics import classification_report

pred = loaded_model.predict(X_test)
pred = np.argmax(pred, axis=1)

test_y = np.argmax(y_test, axis=1)

print(test_y.shape, pred.shape)
print(test_y[:5], pred[:5])

(60,) (60,)
[1 1 1 1 1] [0 0 1 1 0]


In [11]:
print(classification_report(test_y, pred))

              precision    recall  f1-score   support

           0       0.58      0.79      0.67        28
           1       0.73      0.50      0.59        32

    accuracy                           0.63        60
   macro avg       0.65      0.64      0.63        60
weighted avg       0.66      0.63      0.63        60

